In [ ]:
# Check how many 

reference_data = ["./286.fna"]



In [ ]:

for p, d in zip(val_pathogens, val_decoys):
    print("Simulating data for patient", patient_id)
    # decide abundance of the pathogens and decoys
    total_species_num = len(p) + len(d)
    species_abundance = np.random.lognormal(0, 1, total_species_num)
    species_abundance = species_abundance / np.sum(species_abundance)
    species_abundance = np.sort(species_abundance)

    # Assign pathogens the lowest abundance
    species_index = 0
    reads = []
    all_species = []
    abundance_ground_truth = {}
    all_species.extend([(s, 'pathogen') for s in p])
    all_species.extend([(s, 'decoy') for s in d])

    # Simulate reads from pathogen genome
    for species in all_species:
        # genome file to sample reads from
        if species[1] == 'pathogen':
            species_name = pathogens_list[species[0]]
            genome_file = os.path.join(reference_dir, "pathogens", species_name + ".fasta")
        else:
            species_name = decoys_list[species[0]]
            genome_file = os.path.join(reference_dir, "commensals", species_name + ".fasta")

        # determine number of reads of this species
        num_reads = int(species_abundance[species_index] * READ_NUM * 4000)

        # Simulate reads
        simulated_read = subprocess.run(["badread", "simulate", "--reference", genome_file, "--quantity", str(num_reads), "--length", "4000,2000", 
                                                                "--glitches", "0,0,0", "--junk_reads", "0", "--random_reads", "0", "--chimeras", "0"], capture_output=True)
        
        # Store the reads in `reads` list
        read_list = simulated_read.stdout.decode("utf-8").split('\n')
        read_list = [r for i, r in enumerate(read_list) if i % 4 == 1]
        reads.extend(read_list)

        # store ground truth
        abundance_ground_truth[species_name] = species_abundance[species_index]

        # move on to next species
        species_index += 1
    
    # Output raw reads
    read_index = np.arange(len(reads))
    random.shuffle(read_index)

    read_i = 0
    with open(os.path.join(output_dir, "patient" + str(patient_id) + ".fasta"), 'w') as f:
        for i, r in enumerate(read_index):
            f.write(">" + str(i) + "\n")
            f.write(reads[r] + "\n")
    
    # Output labels
    with open(os.path.join(output_dir, "patient" + str(patient_id) + "_labels.csv"), 'w') as f:
        f.write("labels\n")
        for species in p:
            f.write(pathogens_list[species] + "\n")
    
    # Output ground truth abundance
    with open(os.path.join(output_dir, "patient" + str(patient_id) + "_abundance.csv"), 'w') as f:
        f.write("species,abundance\n")
        for species in abundance_ground_truth:
            f.write(species + "," + str(abundance_ground_truth[species]) + "\n")
    
    patient_id += 1